In [ ]:
# Imports
from dotenv import load_dotenv
from json import dump
from langchain_huggingface import HuggingFaceEmbeddings
from pathlib import Path
from sklearn.metrics import classification_report, roc_auc_score
from time import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import joblib
import os
import polars as pl
import torch

In [ ]:
# Load environment variables
load_dotenv()

# Get the directory of the current file
__dir__ = Path(os.path.abspath(""))
"""
The directory of the current file
"""

# Load environment variables
DATASET_NAME = os.environ["DATASET_NAME"]
"""
Dataset name
"""

EMBEDDING_MODEL_NAME = os.environ["EMBEDDING_MODEL_NAME"]
"""
Embedding model name
"""

CLASSIFIER_MODEL_NAME = os.environ["CLASSIFIER_MODEL_NAME"]
"""
Classifier model name
"""

# Create the output directory
OUTPUT_DIRECTORY = __dir__ / f"../data/notebooks/classifier-hybrid/{DATASET_NAME.replace("/", "-")}/{EMBEDDING_MODEL_NAME.replace("/", "-")}/{CLASSIFIER_MODEL_NAME.replace("/", "-")}"
OUTPUT_DIRECTORY.mkdir(parents=True, exist_ok=True)

In [ ]:
# Load the embedding model
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    show_progress=True,
)

In [ ]:
# Load the tokenizer and classifier models
dtype = torch.bfloat16
tokenizer = AutoTokenizer.from_pretrained(CLASSIFIER_MODEL_NAME)
llm_model = AutoModelForSequenceClassification.from_pretrained(CLASSIFIER_MODEL_NAME, dtype=dtype)

In [ ]:
# Create the classifier pipeline
llm_classifier = pipeline(
  "text-classification",
  model=llm_model,
  tokenizer=tokenizer,
  truncation=True,
  max_length=512,
  device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
)

In [ ]:
# Load the prepared dataset
test_df = pl.read_parquet(__dir__ / f"../data/notebooks/prepare-datasets/{DATASET_NAME.replace("/", "-")}/test.parquet")

In [ ]:
# Load the classifier
base_classifier = joblib.load(__dir__ / f"../data/notebooks/classifier-embedding/{DATASET_NAME.replace("/", "-")}/{EMBEDDING_MODEL_NAME.replace("/", "-")}/random-forest.joblib")

In [ ]:
MIN_BENIGN_CONFIDENCE = 0.6
MIN_MALICIOUS_CONFIDENCE = 0.6

def hybrid_classify(text: str) -> bool:
  """
  Hybrid classifier
  """

  # Embed the text
  embedding = embedding_model.embed_documents(texts=[text])[0]

  # Classify the embedding
  prediction = base_classifier.predict_proba([embedding])[0]

  # Determine the label based on confidence
  if prediction[0] >= MIN_BENIGN_CONFIDENCE:
    return False # Benign
  elif prediction[1] >= MIN_MALICIOUS_CONFIDENCE:
    return True # Malicious
  else:
    # Adjudicate with the LLM
    return llm_classifier(text)[0]["label"] != llm_model.config.id2label[0]

In [ ]:
# Benchmark the classifier
start = time()
y_predictions = test_df["text"].map_elements(hybrid_classify)
end = time()

elapsed_test_classify = end - start

In [ ]:
# Get the Y actual labels
y_actual = test_df["label"].to_list()

In [ ]:
# Save the results
with open(OUTPUT_DIRECTORY / "results.json", "w", encoding="utf-8") as results_file:
  dump({
      "auc": roc_auc_score(y_actual, y_predictions),
      "report": classification_report(y_actual, y_predictions, target_names=["benign", "malicious"], output_dict=True),
      "classify_time": elapsed_test_classify,
      "total_time": elapsed_test_classify,
  }, results_file, indent=2)